# Health Inspections Dashboard

Welcome to our PyData Carolinas tutorial! During this session, we're going to learn how to use the new Jupyter [declarative widgets](https://github.com/jupyter-incubator/declarativewidgets) and [dashboarding](https://github.com/jupyter-incubator/dashboards/wiki) capabilities. To do so, we will to turn this notebook into an interactive Health Inspection dashboard. 

We will start with prepped data from the [Durham Open Data](https://opendurham.nc.gov/page/home/) site and code for showing some of it in a table. We'll then progressively add widgets, arrange them in a grid layout, and deploy what we've built as a standalone web app. When we're done, we'll have a notebook-dashboard that:

1. Shows a table of information about food-serving establishments
2. Shows the location of an establishment when a user clicks a row in the table
3. Filters the table by establishment name when the user enters text
4. Shows a bar chart of health code violations per year for the filtered and selected establishments
5. Shows a sample of inspection comments when the user clicks a bar in the chart

All of the libraries you need to build and deploy the dashboard exist in your tutorial environment. We'll walk through the steps you need to take to implement the above features together. If you get stuck, raise your hand or ask a question in the tutorial Slack channel.

Let's get started! &#x1f680;

In [ ]:
import pandas as pd
import numpy as np
import json
import string
import io
import requests
import declarativewidgets

We have to initialize the declarative widgets extension to make sure it's ready to work in this notebook.

In [ ]:
declarativewidgets.init()

We preprocessed the health inspection data [in a separate notebook](https://github.com/jupyter-resources/tutorial-dashboards-declarativewidgets/blob/master/notebooks/inspection/inspection_data_prep.ipynb) to keep focus on dashboards and widgets in this tutorial. Here, we only need to load three CSV files as DataFrames.

First, we read information about establishments that serve food.

In [ ]:
est = pd.read_csv('https://raw.githubusercontent.com/jupyter-resources/tutorial-dashboards-declarativewidgets/master/data/establishments_20160617.csv', parse_dates=['opened', 'closed'])

In [ ]:
est.head(1)

Second, we read information about inspections and their outcomes.

In [ ]:
ins = pd.read_csv('https://raw.githubusercontent.com/jupyter-resources/tutorial-dashboards-declarativewidgets/master/data/inspections_20160617.csv', parse_dates=['date'])

In [ ]:
ins.head(1)

Finally, we load comments about health code violations identified during inspections.

In [ ]:
vio = pd.read_csv('https://raw.githubusercontent.com/jupyter-resources/tutorial-dashboards-declarativewidgets/master/data/violations_20160617.csv')

In [ ]:
vio.head(1)

We're going add a map, so let's split the latitude/longitude string into lists.

In [ ]:
est['latlon'] = est.latlon.str.split(', ')

In [ ]:
est.latlon.head(1)

We're also going to relate establishments to their inspections and inspections to violations in certain cases. We'll perform these joinds on the proper ID columns up-front and use them where applicable below.

In [ ]:
est_ins = pd.merge(est, ins, left_on='id', right_on='est_id', suffixes=('_est', '_ins'))

In [ ]:
est_ins.head(1)

In [ ]:
est_ins_vio = pd.merge(est_ins, vio, left_on='id_ins', right_on='insp_id', suffixes=('', '_vio'))

In [ ]:
est_ins_vio.head(1)

## Table of Establishments

Alright! We've got resonably clean data. We know a bit about its structure. Let's make some of it visible in our dashboard. 

We don't need to write any code in this section. We'll briefly glance at what's here to get a sense of the general steps involved in binding declarative widgets to functions in our notebook. Then we'll position the table widget in dashboard view and deploy the dashboard as a separate web app to get familiar with the workflow.

### Import table widget

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is='urth-core-import'>

### Write functions

In [ ]:
def filter_by_pattern(df, pattern):
    """Filter a DataFrame so that it only includes rows where the name
    column contains pattern, case-insensitive.
    """
    return df[df.name.str.contains(pattern, case=False)]

In [ ]:
def est_table(pattern=''):
    """Build a DataFrame with basic info about establishments and their 
    latest inspection. Filter by checking if the provided pattern
    appears in the establishment name. Clean up the names a bit and 
    format the dates.     
    """
    df = (est_ins.pipe(filter_by_pattern, pattern)          # match pattern (we'll use this later)
                 .dropna(subset=['final_score'])            # keep rows that have an inspection score
                 .sort_values(['name', 'date'])             # sort by establishmnet name then date
                 .drop_duplicates('id_est', keep='last'))   # keep the most recent inspection score
    # truncate names at 25 characters and make them title case
    df['name'] = df.name.str.slice(stop=25).apply(lambda s: string.capwords(s, ' '))
    # format dates
    df['date'] = df.date.dt.strftime('%Y-%m-%d')
    # select columns and make their names nicer
    return df[['est_id', 'name', 'date', 'grade', 'final_score', 'latlon']].rename(columns={'date': 'latest_inspection'})

### Bind widgets to functions

In [ ]:
%%html
<template is="urth-core-bind">
    <urth-core-function ref="est_table" 
                        arg-pattern="{{pattern}}" 
                        result="{{est_table}}" 
                        limit="1600 "
                        delay="500" 
                        auto>
    </urth-core-function>
    <urth-viz-table datarows="{{ est_table.data }}" 
                    rows-visible="19" 
                    selection="{{est_selection}}" 
                    columns="{{ est_table.columns }}" 
                    selection-as-object>
    </urth-viz-table>
</template>

## Establishments on a Map

The `<urth-viz-table>` above makes the user-selected row available to other widgets. Let's introduce a map widget that shows a marker at the latitude and longitude of the selected establishment. 

We'll import the Google Map widget, put it in the notebook, and update it to show the selected establishment location. Then we'll position the map alongside the table in our dashboard view, and deploy our improved app. 

Note: We don't need to write any new Python here.

### Import map and marker widgets

In [ ]:
%%html
<h4>TODO: add widget imports</h4>

### Bind map to table

In [ ]:
%%html
<template is="urth-core-bind">
    <h4>TODO: bind google map to table selection</h4>
</template>

## Filter By Name

Our table shows all 1200+ establishments in the data set. Let's make it possible for the user to filter what's included.

Here we'll import a text widget, put it in the notebook, and bind it to a Python function that filters the DataFrame shown in the table. Like before, we'll lay out and deploy our dashboard.

### Import paper-input

In [ ]:
%%html
<h4>TODO: add widget imports</h4>

### Bind input field to function

In [ ]:
%%html
<template is="urth-core-bind">
    <h4>TODO: bind to filter_by_pattern</h4>
</template>

## Violations Per Year

We've now got a table that we can filter and a map showing table selections. Let's do some plotting. Here we'll add a bar chart showing the median number of violations per year for select establishments.

To do so, we'll import a bar chart widget, show it in the notebook, and bind it to a Python function that calculates the median number of violations. And (surprise!) we'll add it to our dashboard view.

### Import bar chart

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-bar/urth-viz-bar.html" is='urth-core-import'>

### Write counting functions

In [ ]:
def annual_median_violations(df):
    """Compute the median number of violations per year."""
    return (df.groupby('date')
                       .count()
                       .sort_index()
                       .resample('A', how='median')
                       .comments)

In [ ]:
def median_violation_counts(pattern='', est_id:int=None):
    '''
    Build time series of median violation count per year for establishments
    matching the pattern and (optionally) for a specific establishment ID.
    '''
    df = pd.DataFrame()
    # median violations for the filtered set of establishments
    df['table'] = est_ins_vio.pipe(filter_by_pattern, pattern).pipe(annual_median_violations)
    if est_id is not None:
        # median violations for the single selected establishment
        df['selected'] = est_ins_vio[est_ins_vio.est_id == est_id].pipe(annual_median_violations)
    # make the index a column
    return df.reset_index()

### Bind function to bar chart

In [ ]:
%%html
<template is="urth-core-bind">
    <h4>TODO: show median violations plot</h4>
</template>

## Sample Violation Comments

As a final step, we'll show a sample of comments made by inspectors about health code violations. We'll do this when a user click a bar in the chart above.

This time, we'll write a function to draw an appropriate random sample of comments. Then we'll create our own widget to show them in a list view. 

### Write sampling function

In [ ]:
def sample_comments(dt:str, series:str, pattern='', est_id:int=None):
    """Draw a random sample of violation comments for a selected year
    and series.
    
    Parameters
    ==========
    dt: str
        Datetime selected
    series: str
        Series name selected, 'table' or 'selected'
    pattern: str
        Establishment name filter pattern
    est_id: int
        Selected establishment
    
    Return
    ======
    dict
        comments: list
            Sample of comments
        total: int
            Total comments in the population
        for_year: int
            Year portion of dt
        for_series: str
            Echo of the series parameter
    """
    pass

### Bind selected bar to function, return value to custom widget

In [ ]:
%%html
<template is="urth-core-bind">
    <h4>TODO: show sample of violation comments within the selected year / series</h4>
</template>

## Go Further

We've completed a rather nifty dashboard. But there's so much more we could do. Let's spend the rest of the session adding something new to the dashboard. Pair up with someone sitting near you or go at it on your own. The instructors will wander around and help where needed.

Here are some ideas to get you started, but feel free to come up with something on your own:

1. How do inspection scores vary over time?
    * Plot them using `<urth-viz-line>`.
    * Plot them using matplotlib, bokeh, plot.ly, ... instead.
2. What restaurants are near the conference venue? 
    * Show establishments within the map viewport. Let users click to select one on the map, not just in the table.
    * Add addresses to the table. Hook the filter field to addresses as well.
3. How do establishments compare?
    * Show letter grades for the top and bottom scoring establishments over time.
    * Add two drop down boxes to pick two establishments. Plot violation counts for the two over time.
    * Add a time series of scores to a plot every time a user clicks a row in the table. Add a reset button to clear out the set.
4. How can get more details about a selected establishment?
    * Reverse geocode the latlon of an establishment and show its address.
    * Pull data from other cloud services (e.g., Twitter, Foursquare, Yelp!) and display it.
5. Can you improve the look and feel?
    * Add CSS to style the table.
    * Change the color palette of the plot.
6. *YOUR IDEA HERE* - Do whatever floats your boat!

## For More Information

* [Jupyter Declarative Widgets](http://jupyter-incubator.github.io/declarativewidgets/)
* [Jupyter Dashboards](https://github.com/jupyter-incubator/dashboards/wiki)
* [Adding Declarative Widgets to the Jupyter Notebook](http://blog.ibmjstart.net/2016/04/28/adding-declarative-widgets-to-the-jupyter-notebook/)
* [Deploying Dynamic Dashboards](http://blog.ibmjstart.net/2016/04/26/2822/)
* [Developing Data Products with Jupyter Notebooks and Spark](http://www.spark.tc/developing-data-products-with-jupyter-notebooks-and-spark/)
* [Polymer Element Catalog](https://elements.polymer-project.org/)
* [Deploy Your Own Jupyter Notebook and Dashboard Servers](https://github.com/jupyter-incubator/dashboards_setup/tree/master/docker_deploy)